In [ ]:
import cv2 
import os
import shutil
import numpy as np
from matplotlib import pyplot as plt
%matplotlib qt
from tensorflow.keras.models import load_model
import tensorflow as tf
import tkinter as tk
from tkinter import simpledialog, Button
from collections import OrderedDict
import math

In [3]:
# Active local webcam
camera = cv2.VideoCapture(0)

In [4]:
input_value = ''
def click_Pain():
    global input_value
    input_value = f'Pain'
    window.destroy()
    
def click_Neutral():
    global input_value
    input_value = f'Neutral'
    window.destroy()

In [34]:
files = os.listdir('test_images')
file_number = int(files[len(files)-1][0])

In [35]:
files
file_number


4

In [41]:
# Init base variable 
# List to store images from Image Catcher
images = []
# Current file number to save image

# Get current directory path
current_dir = os.getcwd()
# Check if 'test_iamges' folder exists, if not create folder
if 'test_images' not in os.listdir(path=current_dir):
    os.mkdir('test_images')
    file_number = 0
else:
    files = os.listdir('test_images')
    file_number = int(files[len(files)-1][0])+1


while True:
    # Get video frame
    _, camera_frame = camera.read()
    # Draw rectangle on catched image - show area of proper photo
    image_with_border = cv2.rectangle(camera_frame, (100, 100), (500, 500), (255,0,0), 2)
    # Show image with drawn border
    cv2.imshow("Image Catcher", image_with_border)
    # Wait for 1[s] to take an action
    pressed_key = cv2.waitKey(1)
    # If 'q' button clicked, exit "Image Catcher"
    if pressed_key == 113:
        print('Exited camera window')
        # Close all opend Windows
        cv2.destroyAllWindows()
        break
    # If 'space' button clicked, take an image
    if pressed_key == 32:
        # Get clear frame (without border)
        _, image = camera.read()
        # Append list with taken image of size 500x500
        images.append(image[100:500, 100:500])
        # Input window with question: Neutral or Sad    
        
        # Define window for manual classification
        window = tk.Tk()        
        # Define window for manual classification        
        button_v1 = Button(window, text = "Pain", command=click_Pain, width = 100,)
        button_v2 = Button(window, text = "Neutral", command=click_Neutral,  width = 100)
        button_v1.pack(ipadx=10, ipady=10)
        button_v2.pack(ipadx=10, ipady=10)
        window.mainloop()       
        
        # input_value = simpledialog.askstring(title="Classifie Image", prompt="Neutral or Pain?")
        # Set path to save taken image
        image_name = f'test_images/{file_number}_{input_value}.jpg'
        # Save image 
        cv2.imwrite(image_name, image[100:500, 100:500])
        # Iterate file name by 1
        file_number += 1
        # Print information about saved image
        print("Image saved to list")

Image saved to list
Exited camera window


In [42]:
# Release used device - built-in camera
camera.release()

In [6]:
# input_value = simpledialog.askstring(title="Classifie Image", prompt="Neutral or Pain?")

In [7]:
# Reading images from directory
images = []
images_from_file = os.listdir('test_images')

for img_name in images_from_file:
    img = cv2.imread(f'test_images/{img_name}')     
    images.append(img)

In [18]:
predict_emotion = []
predicted_values = []
images_from_file = os.listdir('test_images')
# Load pretrain model of CNN
model = load_model(os.path.join('models', 'pain_detection_model_VGG16.h5'))
# Loop through images list
for index, img in enumerate(images):
    # Resize image to 256x256 size -> expected size by CNN model
    resized_img = tf.image.resize(img, (224,224))
    # Predict pain image
    predict_emotion.append(model.predict(np.expand_dims(resized_img/255, 0)))
    predicted_values.append(predict_emotion[index][0][0])
    # Show iamges from list in RGB colors
    #cv2.imshow(f'Image_{index}: {predict_emotion[index][0][0]}', img)

# Waiting for key action
cv2.waitKey(0)
# Closing all opend windows with images
cv2.destroyAllWindows()


1/1 [==============================] - 0s 77ms/step


In [9]:
# Print image using matplotlib-pyplot
fig = plt.figure()
len(images)%3 
for i in range(0, len(images)):
    plt.subplot(math.ceil(len(images)/4), 4, i+1)
    plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
    plt.gca().set_title('%s: %.3f' %(images_from_file[i], predicted_values[i]), fontsize=15)
    
fig.tight_layout()
plt.show()

In [10]:
fig_1 = plt.figure()
for index, value in enumerate(predicted_values):
    if (images_from_file[index][-11:-4] == "Neutral" and value < 0.5) or (images_from_file[index][-8:-4] == "Pain" and value > 0.5):    
        plt.scatter(index, np.array(value),  color='green', marker ='s', label='Correct Value')
    else:
        plt.scatter(index, np.array(value),  color='red', marker ='s', label='False Value')
plt.plot(np.arange(0, len(predicted_values),1, dtype = int), [0.5 for i in range(len(predicted_values))], color = 'teal', label='Dividing line at 0.5', linewidth=2)
fig_1.suptitle('Pain prediction values', fontsize=20)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.show()

In [11]:
fig_1 = plt.figure()
plt.scatter(np.arange(0, len(predicted_values),1, dtype = int), np.array(predicted_values),  color='teal', marker ='s', label='Value for each image')
plt.plot(np.arange(0, len(predicted_values),1, dtype = int), [0.5 for i in range(len(predicted_values))], color = 'red', label='Dividing line at 0.5', linewidth=2)
fig_1.suptitle('Pain prediction values', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [39]:
# Delete 'test_images' directory after finshed work - optional to use
# Get current directory path
current_dir = os.getcwd()
shutil.rmtree(os.path.join(current_dir, 'test_images'))
